So the program would take the 2 inputs (these are batch import .csv’s for Symbiota; later on we can do this with the Symbiota output which will have all the dynamically assigned unique identifiers), and ideally produce two sets of output: 

(1) just a small table with numbers of shared among AB, unique to A, unique to B; 
    
    - 1 file for the entire analysis would require the user to “fish out” the most sought out pairs. That seems like a feasible task so 1 file for whole analysis is fine to start.

(2) the corresponding, appropriately labeled data files that could be used further to visualize the occurrences.

#### notes on imput data
This is the 6 (taxonomies) * 319 (occurrences), 1914 row long primary table that has all 39 concepts and identified-to occurrences.

In [1]:
# load in the modules
import pandas as pd
from itertools import combinations
from glob import glob

# set input file name to a variable (presumably in same directory as this script)
fn = "Primary_Occurrence_Table_Final_February_14_2021.csv"

# location of the "addendum" data added after the initial input file.
addtl_dir = "./additional_data/"

output_dir = "./output/"

In [2]:
# read in the dataframe
df = pd.read_csv(fn)

# add in the "addendum" data to the df
for csv in glob(f'{addtl_dir}*.csv'):
    addtl_df = pd.read_csv(csv)
    df = df.append(addtl_df, ignore_index=True)

# collectionCode is being used to hold the unique concepts
concepts = df['collectionCode'].unique().tolist()

# catalogNumber is the ID for each occurence
occurences = df['catalogNumber'].unique().tolist()

In [3]:
# create a container to hold the analysis summary
col_names = ["A_code", "B_code", "A/b", "B/a", "A*B"]
analysis_summary = pd.DataFrame(columns = col_names)

# create a container to hold the overlapping matrix
col_names = [x.replace(":", "-") for x in df["collectionCode"].unique()]
combination_counts = pd.DataFrame(columns = col_names)

# iterate over each pair among all unique concepts
for concept_a, concept_b in combinations(concepts, 2):
    
    a_occurences = df[df["collectionCode"] == concept_a]
    a_code = a_occurences["collectionCode"].unique()[0].replace(":", "-")
    
    b_occurences = df[df["collectionCode"] == concept_b]
    b_code = b_occurences["collectionCode"].unique()[0].replace(":", "-")

    # use merge for inner join. However, this duplicates all columns other than catalogNumber
    # since that data is mostly duplicated, drop the right column names
    # this does leave left column data in the concept specific fields (i.e., scientificName)
    combined_occurences = pd.merge(a_occurences, b_occurences, how='inner',
                                   on=['catalogNumber'],
                                   suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

    # count the overlapping occurences among the two concepts
    overlap_count = combined_occurences.shape[0]
    # if the intersection produced results, produce a csv file
    if overlap_count > 0:
        combined_occurences.dropna(axis='columns', inplace=True)
        # generate a file name based on the combination of "institutionCode"s
        fn = f"{output_dir}{a_code}_{b_code}_merged.csv"
        combined_occurences.to_csv(fn)

    #########
    # generate the pair summary table with counts
    fn = f"{a_code}_{b_code}_summary.csv"
    
    # count A not in B (A/b) & B not in A (B/a)
    a_cat_nums = a_occurences['catalogNumber'].unique().tolist()
    b_cat_nums = b_occurences['catalogNumber'].unique().tolist()

    a_exclusives = len([x for x in a_cat_nums if x not in b_cat_nums])
    b_exclusives = len([x for x in b_cat_nums if x not in a_cat_nums])
    
    # organize a line item for the alaysis-wide summary
    analysis_row_data = {"A_code":f"{a_code} (N={len(a_cat_nums)})",
                         "B_code":f"{b_code} (N={len(b_cat_nums)})",
                         "A/b":a_exclusives,
                         "B/a":b_exclusives,
                         "A*B":overlap_count}

    analysis_summary = analysis_summary.append(analysis_row_data,ignore_index=True)
    ##########
    
    ##########
    #summarize the overlapping combination counts

    combination_counts.loc[a_code, b_code] = overlap_count
    combination_counts.loc[b_code, a_code] = overlap_count
    # add the self intersection values
    combination_counts.loc[a_code, a_code] = a_occurences.shape[0]
    combination_counts.loc[b_code, b_code] = a_occurences.shape[0]

In [4]:
# Re-sort the data such that the interesting stuff is on top
# because we want zeroes at the bottom, create a temporary sort column using multiplication
analysis_summary['sort'] = analysis_summary["A*B"] * analysis_summary["A/b"] * analysis_summary["B/a"]
analysis_summary = analysis_summary.sort_values(["sort", "A*B"], ascending=[False, False])
# drop the sort column
analysis_summary.drop('sort',axis='columns', inplace=True)                                                                      

# wrap up and store the outputs
# display analysis_summary 
display(analysis_summary)

# store the analysis summary as its own csv
analysis_summary.to_csv(f"analysis_summary.csv", index=False)

# display unnormalized data
display(combination_counts)
# store the overlap data as it's own csv
combination_counts.to_csv("concept_overlap_counts.csv")

,A_code,B_code,A/b,B/a,A*B
178,CA02-07 (N=212),WEAK20-23 (N=271),48,107,164
179,CA02-07 (N=212),NS21-34 (N=271),48,107,164
176,CA02-07 (N=212),KA94-01 (N=277),42,107,170
527,FACLUST07-20 (N=59),KA94-01 (N=277),42,260,17
529,FACLUST07-20 (N=59),WEAK20-23 (N=271),48,260,11
...,...,...,...,...,...
721,NS21-37 (N=11),NS21-39 (N=42),11,42,0
726,NS21-38 (N=99),NS21-39 (N=42),99,42,0
731,NS21-39 (N=42),KA94-01 (N=277),42,277,0
733,NS21-39 (N=42),WEAK20-23 (N=271),42,271,0


,KA94-02,KA94-03,KA94-04,KA94-05,CA02-07,CA02-08,FA07-09,FA07-10,FA07-11,FA07-12,...,NS21-33,NS21-35,NS21-36,NS21-37,NS21-38,NS21-39,KA94-01,CA02-06,WEAK20-23,NS21-34
KA94-02,17,0,0,0,17,0,0,0,6,0,...,6,0,0,11,0,0,17,17,11,11
KA94-03,0,153,0,0,153,0,38,0,0,0,...,0,0,153,0,0,0,153,153,153,153
KA94-04,0,0,107,0,0,107,0,22,0,8,...,0,8,0,0,99,0,107,107,107,107
KA94-05,0,0,0,42,42,0,0,0,0,0,...,0,0,0,0,0,42,0,42,0,0
CA02-07,17,153,0,42,212,0,38,0,6,0,...,6,0,153,11,0,42,170,212,164,164
CA02-08,0,0,107,0,0,107,0,22,0,8,...,0,8,0,0,99,0,107,107,107,107
FA07-09,0,38,0,0,38,0,38,0,0,0,...,0,0,38,0,0,0,38,38,38,38
FA07-10,0,0,22,0,0,22,0,22,0,0,...,0,0,0,0,22,0,22,22,22,22
FA07-11,6,0,0,0,6,0,0,0,6,0,...,6,0,0,0,0,0,6,6,0,0
FA07-12,0,0,8,0,0,8,0,0,0,8,...,0,8,0,0,0,0,8,8,8,8
